In [1]:
%load_ext autoreload
%autoreload 2

from spot.utils import *

os.chdir(proj_root())

In [37]:
from spot.function_dataset import data_project_from_dir
from spot.function_decoding import EvalResult

dataset_name = "InferTypes4Py"

strategies = ["non-incr", "caller2callee", "callee2caller", "double-traversal"]

eval_dir = get_eval_dir(
    dataset_name,
    "(implicit_imports, new) model-v7--TrainingConfig(drop_env_types=False, add_implicit_rel_imports=True)",
)
evals: list[EvalResult] = [pickle_load(eval_dir / f"{s}-EvalResult.pkl") for s in strategies]

# load test projects
repos_dir = get_dataset_dir(dataset_name) / "repos" / "test"
test_repo_paths = [f for f in repos_dir.iterdir() if f.is_dir()]
test_projects = pmap(
    data_project_from_dir,
    test_repo_paths,
    desc="Loading test projects",
)
assert len(test_projects) > 0


Loading test projects: 100%|██████████| 3/3 [00:11<00:00,  3.94s/it]


In [38]:
from spot.experiments.utils import (
    apply_sigmap,
    apply_sigmap_and_typecheck,
    count_type_errors,
)
import shutil

bin_paths = [
    None,
    Path("/home/jiayi/Projects/typilus/.venv/bin"),
    Path("/home/jiayi/Projects/type4py/.venv/bin"),
]


def compare_results(proj_id: int):
    ex_project = test_projects[proj_id]
    print("Project:", ex_project.name)
    for s, eval in zip(strategies, evals):
        sigmap = eval.predictions[proj_id].assignments
        workdir = Path("data/mypy_temp")
        shutil.rmtree(workdir, ignore_errors=True)
        workdir.mkdir(exist_ok=True, parents=True)
        check_r = apply_sigmap_and_typecheck(
            ex_project, sigmap, workdir, binary_path=bin_paths[proj_id]
        )
        num_errors = count_type_errors(check_r).total()
        print(f"\t{s} errors:", num_errors)

for i in range(len(test_projects)):
    compare_results(i)


Project: SPOT
	non-incr errors: 585
	caller2callee errors: 532
	callee2caller errors: 482
	double-traversal errors: 478
Project: typilus
	non-incr errors: 401
	caller2callee errors: 267
	callee2caller errors: 232
	double-traversal errors: 238
Project: type4py
	non-incr errors: 163
	caller2callee errors: 158
	callee2caller errors: 147
	double-traversal errors: 155
